In [1]:
import psycopg2
import pandas as pd
import sys
import os

sys.path.insert(1, os.path.join(sys.path[0], '../src/app/utils'))
sys.path.insert(1, os.path.join(sys.path[0], '../src/config/'))
from db_connection import DB
from config import db_config

In [2]:
DATA_FULL_PATH = "D:\\Downloads\\Chrome\\home-credit-default-risk\\"

In [3]:
db = DB(db_config)

###  Загрузка данных

In [4]:
# Для каждого стобца в датафрейме определяем тип данных и создаем список, элемент которого строка "название тип"

def get_cols_types(df):
    def in_bouns(min_val: int, max_val: int, cols_types: list) -> None:
        if max_val <= 32767 and min_val >= -32768:
            cols_types.append('smallint')
        elif max_val <= 2147483647 and min_val >= -2147483648:
            cols_types.append('integer')
        else:
            cols_types.append('bigint')
        
    cols_type = []
    names_types = []
    names = []
    
    for col in df.columns:
        if df[col].dtype == 'object':
            lenght_col = df[col].apply(lambda x: len(x) if not pd.isna(x) else x)
            cols_type.append(f'varchar({int(lenght_col.max())})')
        elif df[col].dtype == 'float64':
            cols_type.append('real')
        else:
            in_bouns(df[col].min(), df[col].max(), cols_type)
            
    for col, types in zip(df.columns, cols_type):
        names_types.append(col + ' ' + types)
        names.append(col)
    return names, names_types

In [5]:
# Бегаем по табличкам, определяем типы данных, подставляем итог в схему запроса

for file in os.listdir(DATA_FULL_PATH):
    table = pd.read_csv(DATA_FULL_PATH + file)
    only_names, names_types = get_cols_types(table)
    only_names = ', '.join(only_names)
    names_types = ', '.join(names_types)
    sql_schema_query = f"""
    DROP TABLE IF EXISTS {file[:len(file) - 4]}; 
    CREATE TABLE {file[:len(file) - 4]}({names_types})
    """
    db.send_sql_query(sql_schema_query)
    print(f'Создали таблицу: {file[:len(file) - 4]}')
    sql_data_query = f"""
    COPY {file[:len(file) - 4]}({only_names})
    FROM '{DATA_FULL_PATH + file}' DELIMITER ',' CSV HEADER;
    """
    db.send_sql_query(sql_data_query)
    print(f'Загрузили данные в таблицу: {file[:len(file) - 4]}')

Создали таблицу: application_train_test
Загрузили данные в таблицу: application_train_test
Создали таблицу: bureau
Загрузили данные в таблицу: bureau
Создали таблицу: bureau_balance
Загрузили данные в таблицу: bureau_balance
Создали таблицу: credit_card_balance
Загрузили данные в таблицу: credit_card_balance
Создали таблицу: installments_payments
Загрузили данные в таблицу: installments_payments
Создали таблицу: POS_CASH_balance
Загрузили данные в таблицу: POS_CASH_balance
Создали таблицу: previous_application
Загрузили данные в таблицу: previous_application


###  Упражнения

In [6]:
# вывести средний доход среди всех клиентов

In [4]:
sql_q = """
SELECT AVG(amt_income_total) as avg_income FROM application_train_test
"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,avg_income
0,170116.05986


In [8]:
# вывести минимальный и максимальный возраст среди всех клиентов

In [9]:
sql_q = """
SELECT MIN(days_birth) / -365 as max_age, MAX(days_birth) / -365 as min_age FROM application_train_test
"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,max_age,min_age
0,69,20


In [10]:
# вывести количество мужчин и женщин

In [11]:
sql_q = """
SELECT COUNT(sk_id_curr), code_gender FROM application_train_test
GROUP BY code_gender
HAVING code_gender='F' or code_gender='M'
"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,count,code_gender
0,235126,F
1,121125,M


In [12]:
# вывести общую сумму, количество и среднюю сумму, запрошенную клиентами в кредит с авто и без

In [13]:
sql_q = """
SELECT  SUM(amt_credit), COUNT(amt_credit), AVG(amt_credit), flag_own_car FROM application_train_test
GROUP BY flag_own_car
"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sum,count,avg,flag_own_car
0,1.304044e+11,235235,554317.381867,N
1,7.900242e+10,121020,652786.569683,Y


In [14]:
# вывести доли клиентов с различным образованием

In [15]:
sql_q = """
SELECT COUNT(sk_id_curr)/356255.0 as ratio, name_education_type FROM application_train_test
GROUP BY name_education_type
"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ratio,name_education_type
0,0.000575,Academic degree
1,0.245271,Higher education
2,0.033687,Incomplete higher
3,0.012045,Lower secondary
4,0.708422,Secondary / secondary special


In [16]:
# подсчитать количество полных лет для клиентов, у которых есть во владении автомобиль и недвижимость.
# вывести топ 10 по возрастанию

In [17]:
sql_q = """
SELECT days_birth / -365 as age, flag_own_car, flag_own_realty FROM application_train_test
WHERE flag_own_car='Y' and flag_own_realty='Y'
ORDER BY age
LIMIT 10
"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,age,flag_own_car,flag_own_realty
0,20,Y,Y
1,20,Y,Y
2,21,Y,Y
3,21,Y,Y
4,21,Y,Y
5,21,Y,Y
6,21,Y,Y
7,21,Y,Y
8,21,Y,Y
9,21,Y,Y


In [18]:
# вывести тех клиентов, у кого доход на одного члена семьи в два раза больше, чем в среднем на одного члена семьи по выборке

In [19]:
sql_q = """
SELECT sk_id_curr, amt_income_total/CNT_FAM_MEMBERS as amt_per_fam_member FROM application_train_test
WHERE amt_income_total/CNT_FAM_MEMBERS > 2 * (SELECT AVG(amt_per_fam_member) FROM (SELECT amt_income_total/CNT_FAM_MEMBERS as amt_per_fam_member FROM application_train_test) as f)
"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr,amt_per_fam_member
0,100002,202500.0
1,100033,270000.0
2,100053,202500.0
3,100070,540000.0
4,100079,225000.0
...,...,...
27728,455940,202500.0
27729,455959,225000.0
27730,456009,225000.0
27731,456119,247500.0


In [20]:
# вывести клиентов старше 60 лет по которым нет данных в bureau

In [21]:
sql_q = """
SELECT application_train_test.sk_id_curr, bureau_ids.sk_id_curr as sk_id_curr_in_bureau FROM application_train_test
FULL JOIN (SELECT DISTINCT sk_id_curr FROM bureau) as bureau_ids
ON application_train_test.sk_id_curr=bureau_ids.sk_id_curr
WHERE days_birth / -365 > 60 AND bureau_ids.sk_id_curr IS NULL
"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr,sk_id_curr_in_bureau
0,100413,None
1,100556,None
2,100857,None
3,101509,None
4,101688,None
...,...,...
5051,443505,None
5052,444278,None
5053,448302,None
5054,449566,None


In [22]:
# вывести женщин, у которых в истории bureau было больше двух кредитов, просроченных на 61 день и более
# отсортировать в порядке убывания по кол-ву таких кредитов

In [23]:
sql_q = """
SELECT application_train_test.sk_id_curr FROM application_train_test
JOIN
    (SELECT sk_id_curr FROM bureau
    JOIN
    (SELECT DISTINCT sk_id_bureau FROM bureau_balance
        WHERE status='3' or status='4' or status='5') as bureau_balance
    ON bureau.sk_id_bureau=bureau_balance.sk_id_bureau) as bureau_over_61
ON application_train_test.sk_id_curr=bureau_over_61.sk_id_curr
WHERE code_gender='F'
GROUP BY application_train_test.sk_id_curr
HAVING COUNT(application_train_test.sk_id_curr) > 2
ORDER BY COUNT(application_train_test.sk_id_curr) DESC
"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr
0,279826
1,246788
2,441227
3,129529
4,164862
...,...
335,119063
336,408404
337,400764
338,258108


In [24]:
# по данным из bureau (БКИ) расчитать долю просрочки в активных займах для каждого клиента
# вывести топ 7 мужчин с наибольшей суммой просрочки, указав для них, помимо прочего,
# сумму активных кредитов и суммы всех кредитов (активных и закрытых).

In [7]:
sql_q = """
SELECT application_train_test.sk_id_curr, sum_all_overdue, sum_active_overdue/NULLIF(sum_active_credit, 0) as ratio, sum_active_credit, sum_all_credits
FROM application_train_test
JOIN
(SELECT sk_id_curr, SUM(amt_credit_sum_overdue) as sum_active_overdue, SUM(amt_credit_sum) as sum_active_credit FROM bureau
    WHERE credit_active='Active'
    GROUP BY sk_id_curr) as bureau
ON application_train_test.sk_id_curr = bureau.sk_id_curr
JOIN
(SELECT sk_id_curr, SUM(amt_credit_sum) as sum_all_credits, SUM(amt_credit_sum_overdue) as sum_all_overdue FROM bureau
    WHERE credit_active='Active' or credit_active='Closed'
    GROUP BY sk_id_curr) as sum_all_credits_and_overdue
ON application_train_test.sk_id_curr = sum_all_credits_and_overdue.sk_id_curr
WHERE code_gender='M'
ORDER BY sum_all_overdue DESC
LIMIT 7

"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr,sum_all_overdue,ratio,sum_active_credit,sum_all_credits
0,435405,3681063.00,0.997578,3690000.0,4054000.5
1,427996,1571697.00,0.371418,4231615.5,5631583.5
2,394113,1332472.50,0.870897,1530000.0,1825801.0
3,266765,1224474.90,0.907018,1350000.0,1421955.0
4,167085,780192.00,4.925314,158404.5,176404.5
5,154595,742491.00,0.749991,990000.0,4245754.5
6,262411,709669.25,0.538240,1318500.0,1773000.0


In [8]:
bureau = db.get_df_from_query("""SELECT * FROM bureau""")

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [10]:
bureau[bureau['sk_id_curr'] == 427996]

,sk_id_curr,sk_id_bureau,credit_active,credit_currency,days_credit,credit_day_overdue,days_credit_enddate,days_enddate_fact,amt_credit_max_overdue,cnt_credit_prolong,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_limit,amt_credit_sum_overdue,credit_type,days_credit_update,amt_annuity
1334102,427996,6600520,Closed,currency 1,-2205,0,-1900.0,-1900.0,NaN,0,43938.0,0.0,0.0,0.0,Consumer credit,-1899,NaN
1334103,427996,6600521,Closed,currency 1,-2348,0,-1983.0,-1983.0,NaN,0,103500.0,0.0,0.0,0.0,Consumer credit,-1983,NaN
1334104,427996,6600523,Closed,currency 1,-1083,0,-840.0,-840.0,NaN,0,600030.0,0.0,0.0,0.0,Consumer credit,-749,NaN
1334105,427996,6600525,Active,currency 1,-481,231,604.0,NaN,NaN,0,199818.0,165681.0,0.0,48010.5,Credit card,-6,NaN
1334106,427996,6600526,Closed,currency 1,-1136,0,-770.0,-754.0,NaN,0,157500.0,0.0,NaN,0.0,Consumer credit,-754,NaN
1334107,427996,6600527,Closed,currency 1,-781,0,-385.0,-476.0,NaN,0,315000.0,0.0,NaN,0.0,Consumer credit,-476,NaN
1334108,427996,6600528,Sold,currency 1,-447,0,283.0,NaN,NaN,0,1800000.0,1329597.0,NaN,1329597.0,Consumer credit,-93,NaN
1334109,427996,6600529,Active,currency 1,-476,199,-111.0,NaN,NaN,0,926797.5,634846.5,NaN,634846.5,Consumer credit,-6,NaN
1334110,427996,6600530,Active,currency 1,-484,178,1342.0,NaN,888840.0,0,3105000.0,2660535.0,NaN,888840.0,Consumer credit,-25,NaN
1614414,427996,6600524,Closed,currency 1,-1567,0,-1202.0,-1262.0,NaN,0,180000.0,0.0,0.0,0.0,Consumer credit,-1262,NaN


###  Фичи

In [5]:
# age - полных лет клиента
# count_credits - количество кредитов
# ratio_overdue - доля задолжностей по кредитам
# amt_per_fam_member - доход на члена семьи
# sum_instalment - суммарное количество досрочных платежей

sql_q = """
SELECT application_train_test.sk_id_curr, count_closed_credits, count_active_credits, amt_credit_sum_overdue/NULLIF(amt_credit_sum, 0) as ratio_overdue, amt_income_total/CNT_FAM_MEMBERS as amt_per_fam_member, sum_instalment 
FROM application_train_test
JOIN
(SELECT sk_id_curr, SUM(amt_credit_sum) as amt_credit_sum, SUM(amt_credit_sum_overdue) as amt_credit_sum_overdue FROM bureau
    GROUP BY sk_id_curr) as bureau
ON application_train_test.sk_id_curr = bureau.sk_id_curr
JOIN
(SELECT sk_id_curr, COUNT(sk_id_curr) as count_active_credits FROM bureau
    WHERE credit_active='Active'
    GROUP BY sk_id_curr) as count_active_credits_col
ON application_train_test.sk_id_curr = count_active_credits_col.sk_id_curr
JOIN
(SELECT sk_id_curr, COUNT(sk_id_curr) as count_closed_credits FROM bureau
    WHERE credit_active='Closed'
    GROUP BY sk_id_curr) as count_closed_credits_col
ON application_train_test.sk_id_curr = count_closed_credits_col.sk_id_curr
JOIN
(SELECT sk_id_curr, SUM(cnt_instalment_future) as sum_instalment FROM POS_CASH_balance
    GROUP BY sk_id_curr) as sum_instalment_table
ON application_train_test.sk_id_curr = sum_instalment_table.sk_id_curr

"""
db.get_df_from_query(sql_q)

D:\PycharmProjects\credit_scoring\notebooks\../src/app/utils\db_connection.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sk_id_curr,count_closed_credits,count_active_credits,ratio_overdue,amt_per_fam_member,sum_instalment
0,100001,4,3,0.0,67500.0,13.0
1,100002,6,2,0.0,202500.0,285.0
2,100003,3,1,0.0,135000.0,162.0
3,100005,1,2,0.0,49500.0,72.0
4,100008,2,1,0.0,49500.0,341.0
...,...,...,...,...,...,...
204127,456247,8,3,0.0,112500.0,122.0
204128,456249,11,2,0.0,112500.0,71.0
204129,456250,1,2,0.0,67500.0,174.0
204130,456253,2,2,0.0,153000.0,34.0
